In [ ]:
# importing libraries 
import pandas as pd
from sklearn.model_selection import train_test_split
import re       #regular expression is used to search the words in a text
import spacy
import nltk 
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords  #stopwords means those words which doesn't add much value to text context like articles
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import numpy as np

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



### Importing dataset

In [ ]:
  #useful in creaing dataframs and storing the data in the dataframes
data  = pd.read_csv("https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true",compression='zip')
data.head(2)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184


### Exploratory Data Analysis

In [ ]:
data['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [ ]:
data['Company'].value_counts()

CITIBANK, N.A.                                                    3226
CAPITAL ONE FINANCIAL CORPORATION                                 2711
BANK OF AMERICA, NATIONAL ASSOCIATION                             2580
JPMORGAN CHASE & CO.                                              2409
WELLS FARGO & COMPANY                                             2001
                                                                  ... 
Time Investment Company, Inc.                                        1
Automotive Services Finance, Inc.                                    1
Foxstar Energy Resources LLC DBA Federal Student Loans Council       1
Uplift, Inc                                                          1
Keystone Credit Services LLC                                         1
Name: Company, Length: 2197, dtype: int64

In [ ]:
# considering usefull data columns (Product,company,Consumer complaint narrative)

complaint_data = data[['Product','Company','Consumer complaint narrative']].rename(columns={'Consumer complaint narrative':'Complaint'})

pd.set_option('display.max_colwidth', -1)
complaint_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


,Product,Company,Complaint
0,Vehicle loan or lease,TRUIST FINANCIAL CORPORATION,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account."
1,Debt collection,CURO Intermediate Holdings,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back."
2,Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times."
3,Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that."
4,Debt collection,"Merchants and Professional Bureau, Inc.","This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. 

We will be encoding our labels using LabelEncoder()

In [ ]:
# Importing label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
complaint_data['Product']= label_encoder.fit_transform(complaint_data['Product'])
  
complaint_data['Product'].unique()

array([5, 2, 0, 3, 1, 4])

In [ ]:
complaint_data

,Product,Company,Complaint
0,5,TRUIST FINANCIAL CORPORATION,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account."
1,2,CURO Intermediate Holdings,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back."
2,5,CAPITAL ONE FINANCIAL CORPORATION,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times."
3,0,CAPITAL ONE FINANCIAL CORPORATION,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that."
4,2,"Merchants and Professional Bureau, Inc.","This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never stopped it from being reported when 

In [ ]:
# Further Exploration on dataset
print(complaint_data.isnull().count())

Product      57453
Company      57453
Complaint    57453
dtype: int64


In [ ]:
print(complaint_data.dtypes)

Product      int64 
Company      object
Complaint    object
dtype: object


### **Data Preprocessing**

In [ ]:
# data set split for further analysis 

train,test = train_test_split(complaint_data,test_size=0.3,random_state=42)

In [ ]:
train.shape

(40217, 3)

In [ ]:
train['Product'].value_counts()

2    15254
1    9237 
3    6826 
0    4910 
4    2071 
5    1919 
Name: Product, dtype: int64

In [ ]:

train['Complaint_processed'] = train['Complaint'].map(lambda x: re.sub('[,\.!?]', '', x))  #(˄ means exclusion of everything which is not alphabets and replacing it with null space)

In [ ]:
train['Complaint_processed'] = train['Complaint_processed'].map(lambda x: x.lower())   #converting all the letters into lowercase
train['Complaint_processed'].head()   

42027    after many weeks of receiving hourly phone calls from navient i sent them a letter instructing them only to contact me by mail per the fair debt collection practices act and the calls stopped although they have emailed me a couple of times today xx/xx/19 the mother of a friend from middle school let me know that navient had called her and left a message for me she is not listed as an emergency or secondary contact in any of my paperwork ; navient should not have her number unless they dig it up themselves this is a huge breach of privacy and a clear violation of the fdcpa given that i had instructed them to only contact me by phone and that they contacted someone completely unrelated to me and told them they were looking for me regarding student loan debt per the fdcpa : " a debt collector may not communicate in connection with the collection of any debt with any person other than the consumer his attorney a consumer reporting agency if otherwise permitted by law the creditor 

In [ ]:
!pip install nltk   #natural language toolkit 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
stop_words = (stopwords.words('english'))

stemmer = PorterStemmer()
stemmer = nltk.stem.SnowballStemmer('english')

## Tokenisation

In [ ]:

def tokenize(text):
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2 and len(re.sub('\d+', '', word.strip('Xx/'))) > 3) ] 
  #  tokens = map(str.lower, tokens)
   stems = [stemmer.stem(item) for item in tokens if (item not in stop_words)]
   return stems


docs = train.Complaint_processed.values.tolist()
data_words = [tokenize(doc) for doc in docs]

In [ ]:
print(len(data_words))

data_words[:2]

40217


[['mani',
  'week',
  'receiv',
  'hour',
  'phone',
  'call',
  'navient',
  'sent',
  'letter',
  'instruct',
  'contact',
  'mail',
  'fair',
  'debt',
  'collect',
  'practic',
  'call',
  'stop',
  'although',
  'email',
  'coupl',
  'time',
  'today',
  'mother',
  'friend',
  'middl',
  'school',
  'know',
  'navient',
  'call',
  'left',
  'messag',
  'list',
  'emerg',
  'secondari',
  'contact',
  'paperwork',
  'navient',
  'number',
  'unless',
  'huge',
  'breach',
  'privaci',
  'clear',
  'violat',
  'fdcpa',
  'given',
  'instruct',
  'contact',
  'phone',
  'contact',
  'someon',
  'complet',
  'unrel',
  'told',
  'look',
  'regard',
  'student',
  'loan',
  'debt',
  'fdcpa',
  'debt',
  'collector',
  'communic',
  'connect',
  'collect',
  'debt',
  'person',
  'consum',
  'attorney',
  'consum',
  'report',
  'agenc',
  'otherwis',
  'permit',
  'creditor',
  'attorney',
  'creditor',
  'attorney',
  'debt',
  'collector',
  'navient',
  'clear',
  'violat',
  'pe

## Converting Text into Numerical Representation


In [ ]:
# In Natural language Processing there are many vectorizer which can convert text tokens to numerical tokens here we will try only two method 
# 1 - TF-IFD vectorizer    
# 2 - Count Vectorizer
# You can  find more about them in Document file

tf_ifd  = TfidfVectorizer(tokenizer=tokenize, max_df = 0.75, min_df=50 , max_features=10000,use_idf= False,  lowercase=False )

# Converting text into numerical representation
cv = CountVectorizer(tokenizer=tokenize, max_df = 0.75, min_df=50 , max_features=10000, lowercase=False)

In [ ]:
tf_vector = tf_ifd.fit_transform(train.Complaint_processed)
vc_vectore = cv.fit_transform(train.Complaint_processed)

In [ ]:
print("tf_vector matrx",tf_vector.A)
tf_vector.shape

tf_vector matrx [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


(40217, 2602)

In [ ]:
print("count vectorizer matrix ",vc_vectore.A)
vc_vectore.shape

count vectorizer matrix  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


(40217, 2602)

In [ ]:
cv_voc = (cv.get_feature_names())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
tf_voc = tf_ifd.get_feature_names()

In [ ]:
print("length of tf_ifd vector vocabalury is :",len(tf_ifd.get_feature_names()))
print("length of cv vector vocabalury is :",len(cv.get_feature_names()))

length of tf_ifd vector vocabalury is : 2602
length of cv vector vocabalury is : 2602


**Now we will begin topic modelling with our predefined 3 techniques**

## Latent Dirichlet Allocation (LDA)



In [ ]:
# Code to Measure time taken by program to execute.
import time
  
# store starting time
begin = time.time()

In [ ]:
# The accurate number of topics are 6
n_topics = 6
# Defining LDA (Latent Dirichlet Allocation) model
model_lda = LDA(n_components = n_topics, learning_method='online',max_iter = 20,learning_offset=50,n_jobs = -1, random_state = 42)
# fit transform on model on our count_vectorizer : running this will return our topics 
X_topics = model_lda.fit_transform(tf_vector)


In [ ]:

# .components_ gives us our topic distribution 
topic_words = model_lda.components_

In [ ]:
X_topics

array([[0.02136762, 0.02136858, 0.02136835, 0.42225855, 0.49199262,
        0.02164428],
       [0.01484049, 0.01481741, 0.01481724, 0.01511957, 0.01502511,
        0.92538019],
       [0.02182757, 0.02182845, 0.02182825, 0.89012524, 0.02224825,
        0.02214224],
       ...,
       [0.02915273, 0.0291532 , 0.02915308, 0.02954041, 0.0295885 ,
        0.85341209],
       [0.0271326 , 0.0271331 , 0.02713298, 0.53947875, 0.35161234,
        0.02751024],
       [0.0259567 , 0.02595738, 0.02595721, 0.86986039, 0.02610012,
        0.0261682 ]])

In [ ]:
topic_words

array([[ 0.16670913,  0.16692613,  0.16705338, ...,  0.16695529,
         0.16857241,  0.16715762],
       [ 0.16672667,  0.16703509,  0.16721561, ...,  0.16704648,
         0.16699166,  0.16730447],
       [ 0.16672261,  0.16700981,  0.16717783, ...,  0.16702036,
         0.16698113,  0.16725995],
       [ 0.16666699,  2.00586491,  0.305538  , ...,  2.12090354,
        12.80046577,  3.98454288],
       [ 2.56646553,  3.54810926,  6.2331482 , ...,  1.07530193,
         7.26064341,  0.29522291],
       [ 0.16666696,  1.87197569,  1.2504885 , ...,  2.15252172,
        19.24701147,  1.19430836]])

So, **what do *X_topics* and *topic_words* represents ?**


*  **X_topics**: represents the probability of each topic to be assigned to each document. Thus, its shape would be *(no. of doc , no. of topics)*.It is known as Document-topic matrix.
*   **topic_words**: represents the probability of a particualr word being in aparticular topic. Thus, its shape would be *(no. of topics, no. of words)*.It is known as Topic-Word matrix.



In [ ]:
# Verifying the shapes
print(topic_words.shape)
print(X_topics.shape)

(6, 2602)
(40217, 6)


In [ ]:
n_top_words = 10
topics = []

for i, topic_dist in enumerate(topic_words):
    
    # np.argsort to sorting an array or a list or the matrix acc to their values
    sorted_topic_dist = np.argsort(topic_dist)
    
    # Next, to view the actual words present in those indexes we can make the use of the vocab created earlier
    topic_words = np.array(tf_voc)[sorted_topic_dist]
    
    # so using the sorted_topic_indexes we are extracting the words from the vocabulary
    # obtaining topics + words
    # this topic_words variable contains the Topics  as well as the respective words present in those Topics
    topic_words = topic_words[:-n_top_words:-1]
    topics.append(topic_words)
    print ("Topic", str(i), topic_words)
topics=np.array(topics) 

Topic 0 ['usaa' 'attatch' 'permanet' 'temporari' 'destroy' 'unresolv' 'hide'
 'inspector' 'any/al']
Topic 1 ['max' 'rebil' 'redact' 'statu' 'dual' 'transunion' 'loop' 'biggest'
 'sleep']
Topic 2 ['exhibit' 'testimoni' 'creditworthi' 'manual' 'mandat' 'authent' 'carri'
 'critic' 'non-compli']
Topic 3 ['payment' 'loan' 'call' 'mortgag' 'month' 'would' 'time' 'receiv'
 'compani']
Topic 4 ['debt' 'credit' 'report' 'account' 'collect' 'compani' 'inform' 'receiv'
 'letter']
Topic 5 ['account' 'card' 'call' 'credit' 'bank' 'charg' 'receiv' 'time' 'would']


In [ ]:
topics

array([['usaa', 'attatch', 'permanet', 'temporari', 'destroy',
        'unresolv', 'hide', 'inspector', 'any/al'],
       ['max', 'rebil', 'redact', 'statu', 'dual', 'transunion', 'loop',
        'biggest', 'sleep'],
       ['exhibit', 'testimoni', 'creditworthi', 'manual', 'mandat',
        'authent', 'carri', 'critic', 'non-compli'],
       ['payment', 'loan', 'call', 'mortgag', 'month', 'would', 'time',
        'receiv', 'compani'],
       ['debt', 'credit', 'report', 'account', 'collect', 'compani',
        'inform', 'receiv', 'letter'],
       ['account', 'card', 'call', 'credit', 'bank', 'charg', 'receiv',
        'time', 'would']], dtype='<U18')

In [ ]:
doc_topic = model_lda.transform(tf_vector)  
topic_to_doc = []

# Iterating through every value
for n in range(doc_topic.shape[0]):    
    # argmax() gives maximum index value
    topic_doc = doc_topic[n].argmax()
    topic_to_doc.append(topic_doc)

def doc_to_topic(doc_no,topic_to_doc=topic_to_doc):
  print("topic assigned to document ",doc_no," is ",topic_to_doc[doc_no+1]," that is ",topics[topic_to_doc[doc_no+1]])

doc_to_topic(2)

topic assigned to document  2  is  5  that is  ['account' 'card' 'call' 'credit' 'bank' 'charg' 'receiv' 'time' 'would']


In [ ]:
topic_assigned_to_doc = pd.DataFrame(X_topics,columns=['topic0','topic1','topic2','topic3','topic4','topic5'])
topic_assigned_to_doc['topic_assigned'] = topic_to_doc

In [ ]:
topic_assigned_to_doc

,topic0,topic1,topic2,topic3,topic4,topic5,topic_assigned
0,0.021368,0.021369,0.021368,0.422259,0.491993,0.021644,4
1,0.014840,0.014817,0.014817,0.015120,0.015025,0.925380,5
2,0.021828,0.021828,0.021828,0.890125,0.022248,0.022142,3
3,0.021853,0.021853,0.021853,0.022105,0.022080,0.890256,5
4,0.063320,0.063329,0.063327,0.063463,0.682671,0.063892,4
...,...,...,...,...,...,...,...
40212,0.020184,0.020184,0.020184,0.898607,0.020371,0.020470,3
40213,0.026347,0.026348,0.026348,0.026583,0.510389,0.383985,4
40214,0.029153,0.029153,0.029153,0.029540,0.029588,0.853412,5
40215,0.027133,0.027133,0.027133,0.539479,0.351612,0.027510,3


In [ ]:
topic_assigned_to_doc['topic_assigned'].value_counts()

5    16011
3    13681
4    10515
0    10   
Name: topic_assigned, dtype: int64

In [ ]:
time.sleep(1)
# store end time
end = time.time()

lda_time = end-begin
# total time taken
print(f"Total runtime of the program is {lda_time}")

Total runtime of the program is 265.3589074611664


In [ ]:
from sklearn.metrics import silhouette_score
silhouette_avg_lda = silhouette_score(X_topics, topic_assigned_to_doc['topic_assigned'])  

In [ ]:
silhouette_avg_lda

0.7868561615030814

In [ ]:
from sklearn.metrics.cluster import homogeneity_score
  
# Evaluate the score
hscore_lda = homogeneity_score(train['Product'], topic_assigned_to_doc['topic_assigned'])
  
print(hscore_lda)

0.3172841426933498


## Non-negative Matrix Factorization (NNMF)

In [ ]:
# store starting time
begin = time.time()

In [ ]:
from sklearn.decomposition import NMF


In [ ]:
# Using TFID vectorizer
tf_ifd  = TfidfVectorizer(tokenizer=tokenize, max_df = 0.75, min_df=50 , max_features=10000,use_idf= False,  lowercase=False )

X = tf_ifd.fit_transform(train.Complaint)
words = np.array(tf_ifd.get_feature_names())

print(X)
print("X = ", words)

  (0, 75)	0.07332355751067665
  (0, 1395)	0.07332355751067665
  (0, 2582)	0.07332355751067665
  (0, 1865)	0.07332355751067665
  (0, 1123)	0.07332355751067665
  (0, 1699)	0.21997067253202995
  (0, 323)	0.21997067253202995
  (0, 1516)	0.2932942300427066
  (0, 2084)	0.07332355751067665
  (0, 1320)	0.07332355751067665
  (0, 1213)	0.1466471150213533
  (0, 485)	0.36661778755338326
  (0, 1382)	0.07332355751067665
  (0, 879)	0.07332355751067665
  (0, 583)	0.36661778755338326
  (0, 416)	0.1466471150213533
  (0, 1739)	0.07332355751067665
  (0, 2227)	0.07332355751067665
  (0, 105)	0.07332355751067665
  (0, 762)	0.07332355751067665
  (0, 529)	0.07332355751067665
  (0, 2375)	0.07332355751067665
  (0, 2380)	0.07332355751067665
  (0, 1497)	0.07332355751067665
  (0, 983)	0.07332355751067665
  :	:
  (40216, 209)	0.10369516947304253
  (40216, 247)	0.10369516947304253
  (40216, 1866)	0.10369516947304253
  (40216, 1018)	0.10369516947304253
  (40216, 869)	0.10369516947304253
  (40216, 1469)	0.1036951694730

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Applying Non-Negative Matrix Factorization over 6 topics
 
model_nmf = NMF(n_components=6, solver="mu")
W = model_nmf.fit_transform(X)
H = model_nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-10:]]])))
     print()

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


Topic 1: back,number,phone,compani,they,time,told,would,receiv,call

Topic 2: ident,remov,this,inform,compani,account,collect,credit,report,debt

Topic 3: fund,balanc,deposit,money,charg,open,close,check,bank,account

Topic 4: statement,loan,mortgag,make,report,made,credit,late,month,payment

Topic 5: never,fraud,disput,use,bank,balanc,purchas,charg,credit,card

Topic 6: would,document,request,compani,interest,year,home,servic,mortgag,loan



In [ ]:
doc_topic_byNMF = model_nmf.transform(X)  
topic_to_doc_byNMF = []
# iterating over ever value till the end value
for n in range(doc_topic_byNMF.shape[0]):
    
    # argmax() gives maximum index value
    topic_doc = doc_topic_byNMF[n].argmax()
    topic_to_doc_byNMF.append(topic_doc)

def doc_to_topic_byNMF(doc_no,topic_to_doc_byNMF=topic_to_doc_byNMF):
  print("topic assigned to document ",doc_no," is ",topic_to_doc_byNMF[doc_no+1]," that is ",topics[topic_to_doc_byNMF[doc_no+1]])

doc_to_topic_byNMF(2)

topic assigned to document  2  is  2  that is  ['exhibit' 'testimoni' 'creditworthi' 'manual' 'mandat' 'authent' 'carri'
 'critic' 'non-compli']


In [ ]:
topic_assigned_to_doc_byNMF = pd.DataFrame(W,columns=['topic0','topic1','topic2','topic3','topic4','topic5'])
topic_assigned_to_doc_byNMF['topic_assigned'] = topic_to_doc_byNMF

In [ ]:
topic_assigned_to_doc_byNMF

,topic0,topic1,topic2,topic3,topic4,topic5,topic_assigned
0,0.042731,0.037979,0.000000,0.000000,0.000000,0.000000,0
1,0.053830,0.000000,0.000000,0.000000,0.064096,0.015837,4
2,0.069310,0.000000,0.000000,0.000000,0.015801,0.000000,0
3,0.026675,0.000000,0.068048,0.000000,0.000000,0.000000,2
4,0.000070,0.063823,0.000000,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...
40212,0.053036,0.000000,0.000000,0.000000,0.000617,0.061991,5
40213,0.014769,0.006768,0.055458,0.000000,0.000000,0.017455,2
40214,0.005716,0.007677,0.029752,0.014134,0.100954,0.000888,4
40215,0.034629,0.000000,0.000000,0.000000,0.000000,0.006125,0


In [ ]:
topic_assigned_to_doc_byNMF['topic_assigned'].value_counts()

0    8789
1    8707
2    7159
4    5519
3    5074
5    4969
Name: topic_assigned, dtype: int64

In [ ]:
time.sleep(1)
# store end time
end = time.time()

nmf_time = end-begin
# total time taken
print(f"Total runtime of the program is {nmf_time}")

Total runtime of the program is 158.20977067947388


In [ ]:
from sklearn.metrics import silhouette_score
silhouette_avg_nmf = silhouette_score(X_topics, topic_assigned_to_doc_byNMF['topic_assigned']) 
silhouette_avg_nmf

-0.07932799040112454

In [ ]:
from sklearn.metrics.cluster import homogeneity_score
  
# Evaluate the score
hscore_nnmf = homogeneity_score(train['Product'], topic_assigned_to_doc_byNMF['topic_assigned'])
  
print(hscore_nnmf)

0.30896245131965394


## Latent Semantic Analysis (LSA)

In [ ]:
from scipy import linalg, spatial
from sklearn.decomposition import PCA, SparsePCA, TruncatedSVD
from sklearn.feature_extraction.text import (CountVectorizer, TfidfTransformer, TfidfVectorizer)

from sklearn.utils.extmath import randomized_svd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# store starting time
begin = time.time()

In [ ]:
from sklearn.cluster import KMeans

X = tf_ifd.fit_transform(train.Complaint)
words = np.array(tf_ifd.get_feature_names())

num_clusters = 6

km = KMeans(n_clusters=num_clusters)
km.fit(X)
    
clusters = km.labels_.tolist()
print(clusters)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[2, 4, 5, 0, 2, 5, 5, 4, 0, 1, 1, 2, 5, 1, 1, 5, 0, 4, 2, 4, 0, 3, 1, 1, 5, 5, 2, 4, 2, 5, 5, 2, 5, 3, 5, 4, 1, 3, 2, 3, 5, 2, 3, 2, 0, 2, 4, 0, 5, 3, 1, 5, 4, 2, 0, 4, 1, 2, 4, 3, 1, 0, 5, 5, 3, 4, 3, 2, 2, 5, 5, 5, 2, 4, 5, 5, 1, 5, 5, 5, 3, 5, 1, 5, 0, 0, 2, 5, 4, 2, 3, 5, 4, 3, 1, 5, 1, 5, 2, 3, 3, 0, 3, 1, 5, 3, 5, 3, 2, 1, 1, 5, 5, 1, 1, 5, 4, 2, 3, 5, 2, 0, 5, 0, 4, 2, 1, 0, 4, 0, 3, 1, 3, 5, 4, 1, 1, 5, 3, 3, 0, 2, 5, 5, 5, 5, 5, 0, 3, 5, 4, 4, 2, 5, 1, 2, 3, 4, 4, 0, 4, 1, 0, 2, 1, 5, 1, 1, 1, 0, 4, 0, 5, 5, 1, 2, 5, 0, 5, 2, 5, 5, 0, 0, 0, 4, 4, 2, 5, 1, 1, 2, 5, 0, 1, 4, 5, 5, 3, 5, 1, 3, 5, 2, 3, 4, 2, 0, 4, 4, 3, 4, 2, 1, 1, 2, 4, 4, 0, 3, 5, 0, 0, 0, 4, 3, 5, 1, 5, 0, 3, 2, 3, 4, 4, 1, 3, 1, 3, 0, 2, 3, 5, 5, 5, 5, 2, 5, 0, 5, 1, 0, 5, 5, 5, 2, 4, 0, 5, 5, 3, 2, 5, 2, 3, 0, 1, 1, 4, 5, 5, 4, 5, 2, 3, 1, 3, 4, 0, 5, 5, 5, 3, 3, 5, 0, 3, 5, 3, 5, 0, 3, 3, 0, 5, 1, 1, 0, 2, 4, 5, 2, 2, 5, 1, 2, 4, 5, 0, 3, 5, 3, 5, 5, 5, 1, 1, 5, 5, 3, 3, 1, 1, 4, 5, 1, 2, 2, 2, 4, 2, 3, 5, 

In [ ]:
U, Sigma, VT = randomized_svd(X, n_components=6, n_iter=100, random_state=122)

svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)
    
print(U.shape) 

for i, comp in enumerate(VT):
    terms_comp = zip(words, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Cluster "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
    print(" ")

(40217, 6)
Cluster 0: 
account
credit
call
payment
report
debt
receiv
 
Cluster 1: 
credit
report
debt
account
collect
ident
disput
 
Cluster 2: 
account
bank
card
check
close
open
charg
 
Cluster 3: 
payment
credit
report
loan
late
month
mortgag
 
Cluster 4: 
card
credit
charg
call
purchas
told
use
 
Cluster 5: 
loan
bank
mortgag
charg
servic
card
document
 


In [ ]:
topic_assigned_to_doc_byLSA = km.fit_predict(X)
topic_assigned_to_doc_byLSA

array([5, 0, 4, ..., 0, 4, 4], dtype=int32)

In [ ]:
topic_assigned_to_doc_byLSA.shape

(40217,)

In [ ]:
time.sleep(1)
# store end time
end = time.time()

lsa_time = end-begin
# total time taken
print(f"Total runtime of the program is {lsa_time}")

Total runtime of the program is 190.9186828136444


In [ ]:
from sklearn.metrics import silhouette_score
silhouette_avg_lsa = silhouette_score(X, topic_assigned_to_doc_byLSA) 
silhouette_avg_lsa

0.029534192477011723

In [ ]:
from sklearn.metrics.cluster import homogeneity_score
  
# Evaluate the score
hscore_lsa = homogeneity_score(train['Product'], topic_assigned_to_doc_byLSA)
  
print(hscore_lsa)

0.29253848541046396


# Comparison



Silhouette Scores

In [ ]:
print(f"The Silhoutte Score for LDA is {silhouette_avg_lda}")
print(f"The Silhoutte Score for NNMF is {silhouette_avg_nmf}")
print(f"The Silhoutte Score for LSA is {silhouette_avg_lsa}")

The Silhoutte Score for LDA is 0.7868561615030814
The Silhoutte Score for NNMF is -0.07932799040112454
The Silhoutte Score for LSA is 0.029534192477011723


Run Time

In [ ]:
print(f"Total runtime of LDA is {lda_time}")
print(f"Total runtime of NNMF is {nmf_time}")
print(f"Total runtime of LSA is {lsa_time}")

Total runtime of LDA is 265.3589074611664
Total runtime of NNMF is 158.20977067947388
Total runtime of LSA is 190.9186828136444


Homogeneity Score

In [ ]:
print(f"Total Homogeneity Score of LDA is {hscore_lda}")
print(f"Total Homogeneity Score of NNMF is {hscore_nnmf}")
print(f"Total Homogeneity Score of LSA is {hscore_lsa}")

Total Homogeneity Score of LDA is 0.3172841426933498
Total Homogeneity Score of NNMF is 0.30896245131965394
Total Homogeneity Score of LSA is 0.29253848541046396
